In [1]:
from fmm import FastMapMatch,Network,NetworkGraph,UBODTGenAlgorithm,UBODT,FastMapMatchConfig

In [2]:
network = Network("data/stockholm/edges.shp","fid", "u", "v")
print "Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count())
graph = NetworkGraph(network)

Nodes 6974 edges 15021


In [3]:
# Can be skipped if you already generated an ubodt file
ubodt_gen = UBODTGenAlgorithm(network,graph)
# The delta is defined as 3 km approximately. 0.03 degrees. 
status = ubodt_gen.generate_ubodt("data/stockholm/ubodt.txt", 0.03, binary=False, use_omp=True)
# Binary is faster for both IO and precomputation
# status = ubodt_gen.generate_ubodt("stockholm/ubodt.bin", 0.03, binary=True, use_omp=True)
print status

Status: success
Time takes 5.742 seconds



In [3]:
ubodt = UBODT.read_ubodt_csv("data/stockholm/ubodt.txt")
### Create FMM model
fmm_model = FastMapMatch(network,graph,ubodt)

In [4]:
from shapely.geometry import LineString,shape, mapping
import json
from ipyleaflet import GeoJSON
from shapely.wkt import dumps, loads
traj_style = {
  'fillColor':'green',
  'opacity':1.0,
  'weight':10
}
mr_style = {
   'fillColor':'orange',
   'opacity':0.8,
   'weight':10
} 
def match_geojson_network(traj, k, radius, gps_error):
    traj_wkt = shape(traj["geometry"]).wkt
    r_degree = radius/1.1e5
    e_degree = gps_error/1.1e5
    fmm_config = FastMapMatchConfig(k,r_degree,e_degree)
    result = fmm_model.match_wkt(traj_wkt, fmm_config)
    mr_wkt = result.mgeom.export_wkt()
    if len(list(result.cpath))>0:
        return GeoJSON(name="Matched Traj",
                          data=mapping(loads(mr_wkt)), 
                           style = traj_style)
    else:
        return None
def traj_geojson_layer(traj):
    return GeoJSON(name="Traj",
                      data=traj, 
                       style = mr_style)

### Interactive draw and matching

In [5]:
from ipyleaflet import Map, basemaps, WidgetControl, DrawControl
from ipywidgets import IntSlider, ColorPicker, jslink, FloatSlider, Button, Layout, Label

# Add widget

m = Map(center=(59.341884644077, 18.06016188114882), zoom=14, 
        basemap=basemaps.OpenStreetMap.Mapnik)

k_slider = IntSlider(description='k:', min=4, max=32, value=20,step=4)
k_widget_control = WidgetControl(widget=k_slider, position='topright')
m.add_control(k_widget_control)

r_slider = IntSlider(description='r (m):', min=50, max=500, value=300, step =50)
r_widget_control = WidgetControl(widget=r_slider, position='topright')
m.add_control(r_widget_control)

e_slider = IntSlider(description='e (m):', min=50, max=200, value=140, step = 30)
e_widget_control = WidgetControl(widget=e_slider, position='topright')
m.add_control(e_widget_control)

draw_control = DrawControl(circlemarker={}, polygon={}, edit=False,
    remove=False)

draw_control.polyline =  {
    "shapeOptions": {
        "color": "#fca45d",
        "weight": 6,
        "opacity": 1
    }
}

m.add_control(draw_control)

clear_map_btn = Button(
    description='',
    icon = 'fa-trash',
    layout = Layout(width = '28px',padding='0px')
)

label_widget= Label(layout = Layout(width = '300px',padding='0px'))
label_widget.value = "Match status: {} ".format("  -  ") 
label_control = WidgetControl(widget=label_widget, position='topright',max_width = 200)
m.add_control(label_control)

clear_map_btn_control = WidgetControl(widget=clear_map_btn, position='topleft',max_width=32)
m.add_control(clear_map_btn_control)

### Add event listensers

mr_layer = None
traj_layer = None

def clear_map():
    global traj_layer, mr_layer
    if (mr_layer!=None):
        m.remove_layer(mr_layer) 
        mr_layer = None
    if (traj_layer!=None):
        m.remove_layer(traj_layer) 
        traj_layer = None
    label_widget.value = "Match status: {} ".format("-")     

def update_map(data):
    global traj_layer, mr_layer
    if data==None:
        if traj_layer!=None:
            m.remove_layer(traj_layer)
            if (mr_layer!=None):
                m.remove_layer(mr_layer)
                mr_layer = None
            temp_mr_layer = match_geojson_network(traj_layer.data,
                k_slider.value,r_slider.value,e_slider.value)
            if (temp_mr_layer!=None):
                label_widget.value = "Match status: {} ".format("True")
                m.add_layer(temp_mr_layer)
                mr_layer = temp_mr_layer
            else:
                label_widget.value = "Match status: {} ".format("False") 
            m.add_layer(traj_layer)
        else:
            label_widget.value = "Match status: {} ".format("-") 
    else:
        temp_mr_layer = match_geojson_network(data,
            k_slider.value,r_slider.value,e_slider.value)
        if temp_mr_layer!=None:
            label_widget.value = "Match status: {} ".format("True") 
            if (mr_layer!=None):
                m.remove_layer(mr_layer)
                mr_layer = None
            temp_traj_layer = traj_geojson_layer(data)    
            if (traj_layer!=None):
                m.remove_layer(traj_layer)
                traj_layer=None
            m.add_layer(temp_mr_layer)
            m.add_layer(temp_traj_layer)
            mr_layer = temp_mr_layer
            traj_layer = temp_traj_layer
        else:
            label_widget.value = "Match status: {} ".format("False") 
            temp_traj_layer = traj_geojson_layer(data)    
            if (traj_layer!=None):
                m.remove_layer(traj_layer)
                traj_layer = None
            m.add_layer(temp_traj_layer)
            traj_layer = temp_traj_layer

def on_button_clicked(b):
    clear_map()
def on_value_change(change):
    update_map(None)
def handle_draw(target, action, geo_json):
    global test_json
    if (action=="deleted"):
        m.remove_layer(mr_layer)
        m.remove_layer(traj_layer)
        mr_layer = None
        traj_layer = None
    if (action=="created"):
        draw_control.clear()
        test_json =geo_json
        update_map(geo_json)
        
k_slider.observe(on_value_change, names='value')
r_slider.observe(on_value_change, names='value')
e_slider.observe(on_value_change, names='value')
clear_map_btn.on_click(on_button_clicked)
draw_control.on_draw(handle_draw)
        
m.layout.height= "540px"
m

TWFwKGNlbnRlcj1bNTkuMzQxODg0NjQ0MDc3LCAxOC4wNjAxNjE4ODExNDg4Ml0sIGNvbnRyb2xzPShab29tQ29udHJvbChvcHRpb25zPVt1J3Bvc2l0aW9uJywgdSd6b29tX2luX3RleHQnLCDigKY=


### Match a GeoJSON object

In [6]:
example_geojson = {u'geometry': {u'type': u'LineString', u'coordinates': [[18.030938, 59.343909], [18.041072, 59.335943], [18.048851, 59.341621], [18.05128, 59.346123], [18.069409, 59.341881]]}, u'type': u'Feature', u'properties': {u'style': {u'opacity': 1, u'weight': 6, u'color': u'#fca45d', u'stroke': True, u'clickable': True, u'fill': False}}}

In [7]:
clear_map()
update_map(example_geojson)

Note: Visualize geojson on the map above

### Compare traj and match result side by side

In [8]:
from ipyleaflet import Map, basemaps, basemap_to_tiles
from ipywidgets import Layout,HBox

l = Layout(flex="l",height="540px")

m1 = Map(center=(59.33258,18.0649), zoom=14, 
        basemap=basemaps.OpenStreetMap.Mapnik,layout=l)
m2 = Map(center=(59.33258,18.0649), zoom=14, 
        basemap=basemaps.OpenStreetMap.Mapnik,layout=l)

m1.add_layer(traj_layer)
m2.add_layer(mr_layer)

# jslink((m1, 'zoom'), (m2, 'zoom'))
# jslink((m1, 'center'), (m2, 'center'))
from traitlets import link
# jslink((m1, 'trait_events'), (m2, 'trait_events'))

map_center_link = link((m1, 'center'), (m2, 'center'))
map_zoom_link = link((m1, 'zoom'), (m2, 'zoom'))

# jslink((m1, 'keyboard_pan_offset'), (m2, 'keyboard_pan_offset'))

HBox([m1,m2])

SEJveChjaGlsZHJlbj0oTWFwKGNlbnRlcj1bNTkuMzMyNTgsIDE4LjA2NDldLCBjb250cm9scz0oWm9vbUNvbnRyb2wob3B0aW9ucz1bdSdwb3NpdGlvbicsIHUnem9vbV9pbl90ZXh0JywgdSfigKY=


### More fluent side map

In [26]:
import folium
import folium.plugins

fm = folium.plugins.DualMap(location=(59.33258,18.0649), zoom_start=12)

def traj_style_func(f):
    return traj_style
def mr_style_func(f):
    return mr_style

folium.GeoJson(
    traj_layer.data,
    traj_style_func,
    name='traj'
).add_to(fm.m1)

folium.GeoJson(
    mr_layer.data,
    mr_style_func,
    name='match traj'
).add_to(fm.m2)
fm

/home/cyang/anaconda2/lib/python2.7/site-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning
